In [6]:
from ollama_helper import OllamaChat

chat = OllamaChat()
model = "gpt-oss:20b"

# Full response
print(chat.chat(model=model, messages=[{"role":"user","content":"Hi"}]))

# Streaming response
for part in chat.stream(model=model,
                        messages=[{"role":"user","content":"Tell me a joke"}]):
    print(part, end="", flush=True)


ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [4]:
import requests

try:
    response = requests.get("http://10.255.255.254:11434")
    print(response.json())
except Exception as e:
    print(f"Connection error: {e}")

Connection error: HTTPConnectionPool(host='10.255.255.254', port=11434): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe2c98d78f0>: Failed to establish a new connection: [Errno 111] Connection refused'))
